### 📚 Context

#### 📚 Experiment Tools
- WanDB
 - 시각화..랑 뭐가있다고?

In [2]:
# !pip install wandb

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=0b44695e6abcef28a247f8d68e8f420c746ed4089ea33b3d4406592aa427f82c
  Stored in directory: c:\users\medici\appdata\local\pip\cache\wheels\54\aa\01\724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6488 sha256=a5dcdca5912228e1597df15ca94de4b7b2510974e42c6444b37e4c9f80b02111
  Stored in directory: c:\users\medici\appdata\local\pip\cache\wheels\9f\69\d1\50b39b308a87998eaf5c1d9095e5a5bd2ad98501e2b7936d36
Successfully built promise subprocess32


- json 파일로 관리하면 좋을 것.

In [3]:
sweep_config = {
  "name" : "mdc_sweep",
  "method" : "bayes",
  "parameters" : {
    "max_depth" : {
      "distribution": "int_uniform",
      "min":2,
      "max":15
    },
    "subsample" :{
      "distribution": "uniform",
      "min": 0.5,
      "max": 1.0
    },
    "colsample_bytree":{
      "distribution": "uniform",
      "min": 0.5,
      "max": 1.0
    }
  },
  "metric":{
      "name": "cv_loss",
      "goal": "minimize"
  }
} 
# 딕셔너리안에 딕셔너리가 있는 구조

In [4]:
import os
from os.path import join

import multiprocessing # cpu 몇개인지 체크
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import wandb # 
# 웹 서버 연결
wandb.init()

# optuna 세팅
sweep_id = wandb.sweep(sweep_config, 
                       project="medici wandb test") #프로젝트명 다른 실험일 때 바꿔주는 것 중요.

n_cpus = multiprocessing.cpu_count()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\medici/.netrc


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: p0hwfbf7
Sweep URL: https://wandb.ai/unchae/medici%20wandb%20test/sweeps/p0hwfbf7


In [5]:
BASE_DIR = './data' 

train_path = os.path.join(BASE_DIR, 'train.csv')
test_path = os.path.join(BASE_DIR, 'test.csv')
submission_path = os.path.join(BASE_DIR, 'sample_submission.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

label = data['credit']

In [6]:
# 불필요한 컬럼 제거
data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'],         inplace=True)

In [7]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t == 'O'] 
num_columns = [c for c    in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

Categorical Columns: 
['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']

Numeric Columns: 
['child_num', 'income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'FLAG_MOBIL', 'work_phone', 'phone', 'email', 'family_size', 'begin_month']


#### 라벨 데이터 인코딩

In [8]:
label = label.astype(int) # 꼭 할 필요는 없음.

#### 전처리 프로세스 함수로 작성

In [9]:
# 복붙.
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리
    imputer = SimpleImputer(strategy='most_frequent')
    tmp_x_train[cat_columns] = imputer.fit_transform(tmp_x_train[cat_columns])
    tmp_x_valid[cat_columns] = imputer.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns]  = imputer.transform(tmp_x_test[cat_columns])
    
    # 스케일링
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    # 인코딩
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_x_train[cat_columns])
    
    tmp_x_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]))
    tmp_x_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]))
    tmp_x_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]))
    
    tmp_x_train.drop(columns=cat_columns, inplace=True)
    tmp_x_valid.drop(columns=cat_columns, inplace=True)
    tmp_x_test.drop(columns=cat_columns, inplace=True)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_x_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_x_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test, tmp_x_test_cat], axis=1)
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

### Ensemble

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

def train():
    with wandb.init() as run: # 완디비와 함께 실행하는 것. 항상 인터넷 연결
        params = wandb.config
        
        val_scores = []
        n_splits = 5

        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

        for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
            x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
            x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]

            # 전처리
            x_train, x_valid, x_test = preprocess(x_train, x_valid, test)

            # 모델 정의
            model = XGBClassifier(n_estimators=1000,
                                  max_depth=params['max_depth'],
                                  subsample=params['subsample'],
                                  colsample_bytree=params['colsample_bytree'],
#                                   tree_method='gpu_hist', #gpu있으면 실행. 코랩 가능.
                                  n_jobs=n_cpus-1)

            # 모델 학습
            model.fit(x_train, y_train, 
                      eval_metric='mlogloss', 
                      eval_set=[[x_train, y_train], [x_valid, y_valid]],
                      early_stopping_rounds=100,
                      verbose=100)

            # 훈련, 검증 데이터 log_loss 확인
            trn_logloss = log_loss(y_train, model.predict_proba(x_train))
            val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
            print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))

            val_scores.append(val_logloss)
            
        metrics = {"cv_loss": np.mean(val_scores)}
        wandb.log(metrics)
count = 5
wandb.agent(sweep_id, function=train, count=count)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: hneiuwbl with config:
wandb: 	colsample_bytree: 0.6536476992793909
wandb: 	max_depth: 2
wandb: 	subsample: 0.764610425178516


[0]	validation_0-mlogloss:0.97922	validation_1-mlogloss:0.97823
[100]	validation_0-mlogloss:0.77413	validation_1-mlogloss:0.79073
[200]	validation_0-mlogloss:0.75155	validation_1-mlogloss:0.78436
[300]	validation_0-mlogloss:0.73410	validation_1-mlogloss:0.77968
[400]	validation_0-mlogloss:0.71929	validation_1-mlogloss:0.77480
[500]	validation_0-mlogloss:0.70508	validation_1-mlogloss:0.77010
[600]	validation_0-mlogloss:0.69292	validation_1-mlogloss:0.76812
[700]	validation_0-mlogloss:0.68257	validation_1-mlogloss:0.76491
[800]	validation_0-mlogloss:0.67192	validation_1-mlogloss:0.76237
[900]	validation_0-mlogloss:0.66222	validation_1-mlogloss:0.75899
[999]	validation_0-mlogloss:0.65442	validation_1-mlogloss:0.75662
0 Fold, train logloss : 0.65444, validation logloss : 0.7566
[0]	validation_0-mlogloss:0.97865	validation_1-mlogloss:0.98062
[100]	validation_0-mlogloss:0.77073	validation_1-mlogloss:0.80187
[200]	validation_0-mlogloss:0.74855	validation_1-mlogloss:0.79533
[300]	validation_0-

cv_loss,0.76258
_runtime,112
_timestamp,1628321181
_step,0


cv_loss,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: cumdezwd with config:
wandb: 	colsample_bytree: 0.618215640969904
wandb: 	max_depth: 10
wandb: 	subsample: 0.8685038105121736
wandb: Currently logged in as: unchae (use `wandb login --relogin` to force relogin)


...

cv_loss,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: 3z2xkkas with config:
wandb: 	colsample_bytree: 0.7292639877332963
wandb: 	max_depth: 15
wandb: 	subsample: 0.9889245231389854


[0]	validation_0-mlogloss:0.93040	validation_1-mlogloss:0.96712
[100]	validation_0-mlogloss:0.12441	validation_1-mlogloss:0.81687
[115]	validation_0-mlogloss:0.11147	validation_1-mlogloss:0.83647
0 Fold, train logloss : 0.42554, validation logloss : 0.7138
[0]	validation_0-mlogloss:0.93518	validation_1-mlogloss:0.96936
[100]	validation_0-mlogloss:0.12891	validation_1-mlogloss:0.84655
[114]	validation_0-mlogloss:0.11342	validation_1-mlogloss:0.86913
1 Fold, train logloss : 0.42534, validation logloss : 0.7341
[0]	validation_0-mlogloss:0.96574	validation_1-mlogloss:0.99117
[100]	validation_0-mlogloss:0.12850	validation_1-mlogloss:0.83988
[117]	validation_0-mlogloss:0.10843	validation_1-mlogloss:0.86897
2 Fold, train logloss : 0.42184, validation logloss : 0.7312
[0]	validation_0-mlogloss:0.95898	validation_1-mlogloss:0.98726
[100]	validation_0-mlogloss:0.12695	validation_1-mlogloss:0.83883
[115]	validation_0-mlogloss:0.11052	validation_1-mlogloss:0.86533
3 Fold, train logloss : 0.41834, 

cv_loss,0.7242
_runtime,74
_timestamp,1628321472
_step,0


cv_loss,▁
_runtime,▁
_timestamp,▁
_step,▁


In [10]:
# submit.to_csv('oof_first_submit.csv', index=False)